# AI Agent Swarm for Code Improvement
---
**Author:** Toco Tachibana<br>
**GitHub:** https://github.com/Vero-Ventures/llm-swarm<br>
**Date created:** 2024/05/20<br>
**Last modified:** 2024/05/24<br>

## Overview

This guide provides a walkthrough for using [CrewAI](https://www.crewai.com/)'s Agents and [Ollama](ollama.coms)'s Large Language Models (LLMs) to improve Python code.

**CrewAI** is a framework designed to automate various aspects of software development and maintenance. By using intelligent agents, CrewAI can handle tasks ranging from code review to optimization, significantly reducing the time and effort required from human developers.

**Ollama** offers a suite of local Large Language Models (LLMs) that can be integrated into various applications to provide advanced natural language processing capabilities. These models can be run on local hardware, offering a high degree of customization and privacy compared to cloud-based solutions.

By the end of this guide, you will have a clear understanding of how to set up and utilize these tools to enhance your codebase. The process involves configuring AI agents to analyze, review, and suggest improvements to your Python code.

## Installing Ollama and LLMs

Download and run the Ollama installation script:

In [ ]:
import subprocess
import requests

# Download the Ollama installation script
url = "https://ollama.com/install.sh"
try:
  response = requests.get(url)
  response.raise_for_status()
except requests.RequestException as e:
  print(f"Error downloading the script: {e}")
else:
  subprocess.run(["sh", "-c", response.text], check=True)

Activate Ollama server:

In [ ]:
subprocess.Popen(["ollama", "serve"])

<Popen: returncode: None args: ['ollama', 'serve']>

Download local LLMs (`llama3` and `codellama`):

In [ ]:
subprocess.run(["ollama", "pull", "llama3"], check=True)
subprocess.run(["ollama", "pull", "codellama"], check=True)

CompletedProcess(args=['ollama', 'pull', 'codellama'], returncode=0)

Install required library:

In [ ]:
!pip install --q langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 17.8 MB/s eta 0:00:00


Import library and define local models to use:

In [4]:
from langchain_community.llms import Ollama

llama = Ollama(model="llama3")
code_llama = Ollama(model="codellama")

## Configuring AI Agents

Install required library:

In [5]:
!pip install --q crewai

Import required library:

**Note:** `textwrap.dedent` is used to format multi-line strings



In [23]:
from textwrap import dedent
from crewai import Agent, Task, Crew

Define AI Agents:


*   Senior Python Developer - Refactor the given code
*   Senior QA Enginner - Review refacotred code
*   Software Engineering Manager - Oversee the entire refactoring process





In [33]:
senior_python_developer = Agent(
            role="Senior Python Developer",
            goal=("Refactor software functions to improve efficiency and "
                  "readability"),
            backstory=dedent(
                """
                You are a Senior Python Developer with a deep understanding of
                software design patterns and best practices in Python.
                Your task is to refactor code and add docstring, aiming to
                enhance performance and maintainability without altering the
                core functionality."""
            ),
            allow_delegation=False,
            verbose=True,
            llm=code_llama,
        )

In [34]:
senior_qa_engineer = Agent(
            role="Senior QA Engineer",
            goal="Review refactored code for syntactic and logical correctness",
            backstory=dedent(
                """
                You are a meticulous software quality assurance engineer
                specialized in the post-refactoring review. You scrutinize
                refactored code for syntax errors, possible regressions, and
                maintainability, ensuring the changes do not introduce new bugs.
                """
            ),
            allow_delegation=False,
            verbose=True,
            llm=code_llama,
        )

In [35]:
software_engineering_manager = Agent(
            role="Software Engineering Manager",
            goal=("Oversee the entire refactoring process and confirm "
                  "functionality"),
            backstory=dedent(
                """
                Ensure that every piece of refactored code fulfills its intended
                functionality. You oversee the integration and testing phases,
                ensuring high-quality standards and seamless functionality
                across all modules."""
            ),
            allow_delegation=False,
            verbose=True,
            llm=llama,
        )

## Configuring Tasks

Define Tasks:


*   Refactoring - Refactors the given function, assigned to Senior Python Developer
*   QA Review - Review refacotred code, assigned to Senior QA Engineer
*   Consistency Check - Oversee the entire refactoring process

In [36]:
determin_reqs_task = Task(
        description=dedent(
            """\
            You will read over the given code, identify code issues that needs to be changed for each function, determine it's goal, what needs to be changed in it's logic,
            and find any style errors that should be refactored. determine if it needs to be split into
            multiple functions and if so, define them. Do not write any code, only pass along changes
            that need to be made.

            {code}

            Your final answer must be a list of changes that must be made to the function to improve the code
            and refactor it with proper style. This includes improving variable names, adding documentation and comments, and following good coding
            paradignms. do not include any code.
            """
        ),
        agent=software_engineering_manager,
        expected_output="List of refactoring changes to be made",
    )

In [44]:
refactoring_task = Task(
            description=dedent(
                """
                You will refactor the given code in Python, focusing on
                improving its performance, readability, and maintainability
                while ensuring the core functionality remains unchanged. Follow
                the PEP8 style guide to ensure the code conforms to best
                practices. Once refactored, add a docstring to clarify its
                functionality. review things such as changing variable names to be easier to read.

                Original code:
                ------------
                {code}
                """
            ),
            expected_output=dedent(
                "Refactored full Python code with a docstring.",
            ),
            agent=senior_python_developer,
        )

In [45]:
qa_review_task = Task(
            description=dedent(
                """
                You are helping to review and improve the refactored code
                provided by Senior Python Developer. Check for logic errors,
                syntax errors, missing imports, variable naming/casing,
                mismatched brackets, and security vulnerabilities.
                """
            ),
            expected_output=dedent(
                "QA-reviewed full Python code with a docsting and nothing else."
            ),
            agent=senior_qa_engineer,
        )

## Configuring Crew

Define a crew:

In [46]:
refactoring_crew = Crew(
        agents=[
            senior_python_developer,
            senior_qa_engineer,
            software_engineering_manager,
        ],
        tasks=[
            determin_reqs_task,
            refactoring_task,
            qa_review_task,
        ],
        verbose=2,
    )

2024-05-24 14:24:49,377 - 139936331718784 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


Specify input:

In [47]:
input = {
    "function": dedent(
      """
      def findMin (L,startIndx):
        m = L[startIndx]
        index = startIndx
        for i in range (startIndx,len(L)):
            x = L[i]
            if i < m:
                i = index
                m = x
            else:
                pass
        return (m,index)
      """
    )}

Kick off the process:

In [48]:
result = refactoring_crew.kickoff(input)
print(result)

 [DEBUG]: == Working Agent: Software Engineering Manager
 [INFO]: == Starting Task: You will read over the given code, identify the function that needs to be changed, determine it's goal, what needs to be changed in it's logic,
and find any style errors that should be refactored. determine if it needs to be split into
multiple functions and if so, define them. Do not write any code, only pass along changes
that need to be made.


def findMin (L,startIndx):
  m = L[startIndx]
  index = startIndx
  for i in range (startIndx,len(L)):
      x = L[i]
      if i < m:
          i = index
          m = x
      else:
          pass
  return (m,index)


Your final answer must be a list of changes that must be made to the function to improve the function
and refactor it with proper style. This includes improving variable names, adding documentation and comments, and following good coding
paradignms. do not include any code.



> Entering new CrewAgentExecutor chain...
I now can give a great answe